<a href="https://colab.research.google.com/github/kalusam1/Free-ai-classes-2022/blob/main/Using_Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Done Using Colab**

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving cleaned_test.csv to cleaned_test (1).csv
Saving cleaned_train.csv to cleaned_train (1).csv
User uploaded file "cleaned_test.csv" with length 232096 bytes
User uploaded file "cleaned_train.csv" with length 831027 bytes


In [ ]:
!pip install catboost 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
train , test = pd.read_csv('cleaned_train.csv'), pd.read_csv('cleaned_test.csv')

In [ ]:
train.head()

,Unnamed: 0,VehicleID,Maker,Model,Year,Colour,Distance,Abuja,Ibadan,Lagos,Brand New,Foreign Used,Nigerian Used,z_score_distance,log_distance,power_transform_distance,Amount (Million Naira)
0,0,VHL12546,17,142,2011.0,18,80830.0,1,0,0,0,0,1,-0.135602,11.300103,0.002545,2.2
1,1,VHL18827,19,1204,2012.0,18,125000.0,0,1,0,0,0,1,0.376490,11.736069,0.711677,3.5
2,2,VHL19499,29,1040,2010.0,17,110852.0,0,0,1,0,1,0,0.212463,11.615951,0.504982,9.2
3,3,VHL17991,34,590,2017.0,3,30000.0,1,0,0,0,1,0,-0.724908,10.308953,-1.247567,22.8
4,4,VHL12170,53,659,2002.0,17,125206.0,0,1,0,0,0,1,0.378878,11.737716,0.714574,2.6


In [ ]:
X = train.drop(columns  = ['VehicleID','Amount (Million Naira)'])
y = train['Amount (Million Naira)']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 0)

In [ ]:
# from catboost import CatBoostRegressor
# reg = CatBoostRegressor(random_state = 0)
# reg.fit(X_train, y_train)
# y_preds = reg.predict(X_test)

# from sklearn.metrics import mean_squared_error

# mean_squared_error(y_test,y_preds,squared=False)

In [ ]:
 
 
# sub_file = pd.DataFrame({
#       'VehicleID': test['VehicleID'],
#       'Amount (Million Naira)': reg.predict(test.drop(columns = ['VehicleID']))
#   })

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
xgb = XGBRegressor(random_state = 0)
xgb.fit(X_train,y_train)
xgb_preds = xgb.predict(X_test)
mean_squared_error(y_test,xgb_preds,squared=False)

[20:04:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


10.871109001774826

In [28]:
# hyperparameter tuning
from sklearn.model_selection import GridSearchCV
param_grid = {
    'max_depth': [2, 3, 5, 10, 15],
    'learning_rate': [0.05, 0.1, 0.15, 0.20],
    'min_child_weight': [1, 2, 3, 4]
}

grid = GridSearchCV(xgb, param_grid, scoring='neg_mean_squared_error', n_jobs = -1)
grid.fit(X_train, y_train)
grid.best_params_


[20:24:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


{'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 2}

In [29]:
grid_preds = grid.predict(X_test)
mean_squared_error(y_test,grid_preds, squared=False)

10.390924788180714

In [30]:
sub_file = pd.DataFrame({
      'VehicleID': test['VehicleID'],
      'Amount (Million Naira)': grid.predict(test.drop(columns = ['VehicleID']))
  })

sub_file.to_csv('xgboost_grid.csv', index = False)

In [ ]:
# XGBRegressor(learning_rate = .2, max_depth = 3, min_child_weight = 2)